### Final Project motor task classification using EEG Data

#### Importing necessary libraries

In [ ]:
import numpy as np
import os
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd /content/gdrive/Shareddrives/C247

/content/gdrive/Shareddrives/C247


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten,Dropout
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Reshape
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras import activations

import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from load_data import *
from sklearn.model_selection import KFold
from helpers import *

%load_ext autoreload
%autoreload 2

In [ ]:
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# Loading full data
data_path = './project'

X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test = load_full_data(data_path)

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115,)
Person test shape: (443,)


In [ ]:
## Adjusting the labels to {0,1,2,3}

# left hand - 0
# right hand - 1
# feet motion - 2
# tongue motion - 3

# labels are given as {769, 770, 771, 772}
base = y_train_valid.min()
y_train_valid-= base
y_test-= base

In [ ]:
# Getting data for individual subjects

subject_data_train = split_data_by_subject(X_train_valid, y_train_valid, person_train_valid)
subject_data_test = split_data_by_subject(X_test, y_test, person_test)

In [ ]:
# Changing of data dimensions

print ('Shape of training set: {}'.format(X_train_valid.shape))
print ('Shape of test set: {}'.format(X_test.shape))
print ('Shape of train labels: {}'.format(y_train_valid.shape))
print ('Shape of test labels: {}'.format(y_test.shape))

y_train_cv = to_categorical(y_train_valid, 4)
y_test_cv = to_categorical(y_test, 4)
print('Shape of training labels after categorical conversion:', y_train_cv.shape)
print('Shape of test labels after categorical conversion:', y_test_cv.shape)

# Adding width of the segment to be 1
x_train_cv = X_train_valid.reshape(X_train_valid.shape[0], X_train_valid.shape[1], X_train_valid.shape[2], 1)
x_test_cv = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
print('Shape of training set after adding width info:',x_train_cv.shape)
print('Shape of test set after adding width info:',x_test_cv.shape)


# Reshaping the training and validation dataset
x_train_cv = np.swapaxes(x_train_cv, 1,3)
x_train_cv = np.swapaxes(x_train_cv, 1,2)

x_test_cv = np.swapaxes(x_test_cv, 1,3)
x_test_cv = np.swapaxes(x_test_cv, 1,2)
print('Shape of training set after dimension reshaping:',x_train_cv.shape)

print('Shape of test set after dimension reshaping:',x_test_cv.shape)

Shape of training set: (2115, 22, 1000)
Shape of test set: (443, 22, 1000)
Shape of train labels: (2115,)
Shape of test labels: (443,)
Shape of training labels after categorical conversion: (2115, 4)
Shape of test labels after categorical conversion: (443, 4)
Shape of training set after adding width info: (2115, 22, 1000, 1)
Shape of test set after adding width info: (443, 22, 1000, 1)
Shape of training set after dimension reshaping: (2115, 1000, 1, 22)
Shape of test set after dimension reshaping: (443, 1000, 1, 22)


#Building 2D CNN 

In [ ]:
from project.helpers import *

**Windowing and sampling**

In [ ]:
# Train data
# train_X, val_X, train_y, val_y = train_test_split(x_train_cv, y_train_cv, test_size=0.2, random_state=42)
x_train_SW, y_train_SW, train_idx_SW = sampleWindows(train_X, train_y, 300, 20)

# Validation data
x_val_SW, y_val_SW, val_idx_SW = sampleWindows(val_X, val_y, 300, 20)

# Test
x_test_SW, y_test_SW, test_idx_SW = sampleWindows(x_test_cv, y_test_cv, 300, 20)

In [ ]:
x_train_SW.shape

(33840, 300, 1, 22)

**Basic CNN Model**

In [ ]:
# Building the CNN model using sequential class
tf.keras.backend.clear_session()
CNNModel = Sequential()

# Conv. block 1
CNNModel.add(Conv2D(filters=32, kernel_size=(10,1), padding='same', activation='elu', input_shape=(300,1,22)))
CNNModel.add(MaxPooling2D(pool_size=(3,1), padding='same'))
CNNModel.add(BatchNormalization())
CNNModel.add(Dropout(0.4))

# Conv. block 2
CNNModel.add(Conv2D(filters=64, kernel_size=(10,1), padding='same', activation='elu'))
CNNModel.add(MaxPooling2D(pool_size=(3,1), padding='same'))
CNNModel.add(BatchNormalization())
CNNModel.add(Dropout(0.4))

# Conv. block 3
CNNModel.add(Conv2D(filters=128, kernel_size=(10,1), padding='same', activation='elu'))
CNNModel.add(MaxPooling2D(pool_size=(3,1), padding='same'))
CNNModel.add(BatchNormalization())
CNNModel.add(Dropout(0.4))

# Conv. block 4
CNNModel.add(Conv2D(filters=256, kernel_size=(10,1), padding='same', activation='elu'))
CNNModel.add(MaxPooling2D(pool_size=(3,1), padding='same'))
CNNModel.add(BatchNormalization())
CNNModel.add(Dropout(0.4))

# Output layer with Softmax activation
CNNModel.add(Flatten()) # Flattens the input
CNNModel.add(Dense(4, activation='softmax')) # Output FC layer with softmax activation


# Printing the model summary
CNNModel.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 300, 1, 32)        7072      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 100, 1, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 100, 1, 32)       128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 100, 1, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 1, 64)        20544     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 34, 1, 64)        0

In [ ]:
# Model parameters
learning_rate = 1e-4
epochs = 50
cnn_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
# Compiling the model
learning_rate = 1e-4
# epochs = 50
CNNModel.compile(loss='categorical_crossentropy',
                 optimizer=cnn_optimizer,
                 metrics=['accuracy'])

# Training and validating the model
CNNModel_results = CNNModel.fit(x_train_SW,
             y_train_SW,
             batch_size=256,
             epochs=epochs,
             validation_data=(x_val_SW, y_val_SW), verbose=True)

Epoch 1/50
133/133 [==============================] - 6s 40ms/step - loss: 0.9135 - accuracy: 0.6183 - val_loss: 1.2575 - val_accuracy: 0.4733
Epoch 2/50
133/133 [==============================] - 4s 32ms/step - loss: 0.9088 - accuracy: 0.6185 - val_loss: 1.2360 - val_accuracy: 0.4814
Epoch 3/50
133/133 [==============================] - 4s 32ms/step - loss: 0.9080 - accuracy: 0.6223 - val_loss: 1.2498 - val_accuracy: 0.4740
Epoch 4/50
133/133 [==============================] - 4s 32ms/step - loss: 0.8997 - accuracy: 0.6243 - val_loss: 1.2669 - val_accuracy: 0.4715
Epoch 5/50
133/133 [==============================] - 4s 32ms/step - loss: 0.8932 - accuracy: 0.6263 - val_loss: 1.2456 - val_accuracy: 0.4759
Epoch 6/50
133/133 [==============================] - 4s 31ms/step - loss: 0.8908 - accuracy: 0.6313 - val_loss: 1.2715 - val_accuracy: 0.4767
Epoch 7/50
133/133 [==============================] - 4s 34ms/step - loss: 0.8792 - accuracy: 0.6353 - val_loss: 1.2565 - val_accuracy: 0.4784

In [ ]:
cnn_score = CNNModel.evaluate(x_test_SW, y_test_SW, verbose=0)
print('Test accuracy of the basic CNN model:',cnn_score[1])

Test accuracy of the basic CNN model: 0.5206546187400818


In [ ]:
maxVoting(x_test_SW, y_test_SW, test_idx_SW, x_test_cv, CNNModel)

0.6501128668171557


In [ ]:
cnn_score = CNNModel.evaluate(x_test_SW, y_test_SW, verbose=0)
print('Test accuracy of the basic CNN model:',cnn_score[1])

Test accuracy of the basic CNN model: 0.5206546187400818


**Running on only the first 500 instances**

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(x_train_cv[:, 0:500, :, :], y_train_cv, test_size=0.2, random_state=42)

In [ ]:
# Train data
# train_X, val_X, train_y, val_y = train_test_split(x_train_cv, y_train_cv, test_size=0.2, random_state=42)
x_train_SW, y_train_SW, train_idx_SW = sampleWindows(train_X, train_y, 300, 20)

# Validation data
x_val_SW, y_val_SW, val_idx_SW = sampleWindows(val_X, val_y, 300, 20)

# Test
x_test_SW, y_test_SW, test_idx_SW = sampleWindows(x_test_cv[:, 0:500, :, :], y_test_cv, 300, 20)


**Applying normal CNN**

In [ ]:
# Building the CNN model using sequential class
tf.keras.backend.clear_session()
CNNModel = Sequential()

# Conv. block 1
CNNModel.add(Conv2D(filters=128, kernel_size=(5,1), padding='same', activation='elu', input_shape=(300,1,22)))
CNNModel.add(MaxPooling2D(pool_size=(3,1), padding='same'))
CNNModel.add(BatchNormalization())
CNNModel.add(Dropout(0.5))

# Conv. block 2
CNNModel.add(Conv2D(filters=128, kernel_size=(5,1), padding='same', activation='elu'))
CNNModel.add(MaxPooling2D(pool_size=(3,1), padding='same'))
CNNModel.add(BatchNormalization())
CNNModel.add(Dropout(0.5))

# Conv. block 3
CNNModel.add(Conv2D(filters=128, kernel_size=(5,1), padding='same', activation='elu'))
CNNModel.add(MaxPooling2D(pool_size=(3,1), padding='same'))
CNNModel.add(BatchNormalization())
CNNModel.add(Dropout(0.5))

# Conv. block 4
CNNModel.add(Conv2D(filters=256, kernel_size=(5,1), padding='same', activation='elu'))
CNNModel.add(MaxPooling2D(pool_size=(3,1), padding='same'))
CNNModel.add(BatchNormalization())
CNNModel.add(Dropout(0.5))

# Output layer with Softmax activation
CNNModel.add(Flatten()) # Flattens the input
CNNModel.add(Dense(4, activation='softmax')) # Output FC layer with softmax activation


# Printing the model summary
CNNModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 300, 1, 128)       14208     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 100, 1, 128)      0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 100, 1, 128)      512       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 100, 1, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 1, 128)       82048     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 34, 1, 128)       0

In [ ]:
# Model parameters
learning_rate = 1e-4
epochs = 50
cnn_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
# Compiling the model
# learning_rate = 2e-5
cnn_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
epochs = 80
CNNModel.compile(loss='categorical_crossentropy',
                 optimizer=cnn_optimizer,
                 metrics=['accuracy'])

# Training and validating the model
CNNModel_results = CNNModel.fit(x_train_SW,
             y_train_SW,
             batch_size=128,
             epochs=epochs,
             validation_data=(x_val_SW, y_val_SW), verbose=True)

Epoch 1/80
265/265 [==============================] - 18s 31ms/step - loss: 2.1639 - accuracy: 0.2817 - val_loss: 1.5178 - val_accuracy: 0.2976
Epoch 2/80
265/265 [==============================] - 7s 25ms/step - loss: 1.8994 - accuracy: 0.3105 - val_loss: 1.5075 - val_accuracy: 0.3086
Epoch 3/80
265/265 [==============================] - 7s 25ms/step - loss: 1.7356 - accuracy: 0.3284 - val_loss: 1.3960 - val_accuracy: 0.3505
Epoch 4/80
265/265 [==============================] - 7s 26ms/step - loss: 1.6213 - accuracy: 0.3485 - val_loss: 1.4423 - val_accuracy: 0.3518
Epoch 5/80
265/265 [==============================] - 7s 26ms/step - loss: 1.5247 - accuracy: 0.3674 - val_loss: 1.3723 - val_accuracy: 0.3687
Epoch 6/80
265/265 [==============================] - 7s 25ms/step - loss: 1.4503 - accuracy: 0.3847 - val_loss: 1.3191 - val_accuracy: 0.4015
Epoch 7/80
265/265 [==============================] - 7s 25ms/step - loss: 1.3884 - accuracy: 0.4055 - val_loss: 1.3088 - val_accuracy: 0.410

In [ ]:
maxVoting(x_test_SW, y_test_SW, test_idx_SW, x_test_cv, CNNModel)

0.6501128668171557


In [ ]:
cnn_score = CNNModel.evaluate(x_test_SW, y_test_SW, verbose=0)
print('Test accuracy of the basic CNN model:',cnn_score[1])

Test accuracy of the basic CNN model: 0.6297968626022339


**Applying conv2dPreprocessing on subsamples (Using full 1000 instances)**

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(x_train_cv, y_train_cv, test_size=0.2, random_state=42)

x_train_SW, y_train_SW, train_idx_SW = sampleWindows(train_X, train_y, 600, 20)

# Validation data
x_val_SW, y_val_SW, val_idx_SW = sampleWindows(val_X, val_y, 600, 20)

# Test
x_test_SW, y_test_SW, test_idx_SW = sampleWindows(x_test_cv, y_test_cv, 600, 20)

**Applying 2dpreprocessing**

In [ ]:
x_train_SW = conv2DPrepreprocessing(x_train_SW, 5)

x_val_SW = conv2DPrepreprocessing(x_val_SW, 5)

x_test_SW = conv2DPrepreprocessing(x_test_SW, 5)


In [ ]:
x_train_SW.shape

(33840, 5, 60, 22)

In [ ]:
# Building the CNN model using sequential class
tf.keras.backend.clear_session()
CNN2D = Sequential()

# Conv. block 1
CNN2D.add(Conv2D(filters=32, kernel_size=(3,7), padding='same', input_shape=(5, 60, 22),activation='elu'))
# basic_cnn_model2d.add(tensorflow.keras.layers.PReLU())
CNN2D.add(MaxPooling2D(pool_size=(2,2), padding='same')) # Read the keras documentation
CNN2D.add(BatchNormalization())
CNN2D.add(Dropout(0.5))

# Conv. block 2
CNN2D.add(Conv2D(filters=64, kernel_size=(3,7), padding='same',activation='elu'))
# basic_cnn_model2d.add(tensorflow.keras.layers.PReLU())
CNN2D.add(MaxPooling2D(pool_size=(2,2), padding='same'))
CNN2D.add(BatchNormalization())
CNN2D.add(Dropout(0.5))

# Conv. block 3
CNN2D.add(Conv2D(filters=128, kernel_size=(3,7), padding='same',activation='elu'))
# basic_cnn_model2d.add(tensorflow.keras.layers.PReLU())
CNN2D.add(MaxPooling2D(pool_size=(2,2), padding='same'))
CNN2D.add(BatchNormalization())
CNN2D.add(Dropout(0.5))

# Conv. block 4
CNN2D.add(Conv2D(filters=256, kernel_size=(3,7), padding='same',activation='elu'))
# basic_cnn_model2d.add(tensorflow.keras.layers.PReLU())
CNN2D.add(MaxPooling2D(pool_size=(2,2), padding='same'))
CNN2D.add(BatchNormalization())
CNN2D.add(Dropout(0.5))

# Output layer with Softmax activation
CNN2D.add(Flatten()) # Flattens the input
CNN2D.add(Dense(4, activation='softmax')) # Output FC layer with softmax activation


# Printing the model summary
CNN2D.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 5, 80, 32)         14816     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3, 40, 32)        0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 3, 40, 32)        128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 3, 40, 32)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 40, 64)         43072     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2, 20, 64)        0

In [ ]:
# Model parameters
learning_rate = 1e-4
epochs = 50
cnn_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
# Compiling the model
# learning_rate = 1e-5
cnn_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
epochs = 50
CNN2D.compile(loss=categorical_crossentropy,
                 optimizer=cnn_optimizer,
                 metrics=['accuracy'])

# Training and validating the model
CNN2D_results = CNN2D.fit(x_train_SW,
             y_train_SW,
             batch_size=512,
             epochs=epochs,
             validation_data=(x_val_SW, y_val_SW), verbose=True)

In [ ]:
cnn_score = CNN2D.evaluate(x_test_SW, y_test_SW, verbose=0)
print('Test accuracy of the basic CNN model:',cnn_score[1])

Test accuracy of the basic CNN model: 0.6110609769821167


In [ ]:
maxVoting(x_test_SW, y_test_SW, test_idx_SW, x_test_cv, CNN2D)

0.690744920993228


Using 500 samples

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(x_train_cv[:, 0:500, :, :], y_train_cv, test_size=0.2, random_state=42)

x_train_SW, y_train_SW, train_idx_SW = sampleWindows(train_X, train_y, 250, 20)

# Validation data
x_val_SW, y_val_SW, val_idx_SW = sampleWindows(val_X, val_y, 250, 20)

# Test
x_test_SW, y_test_SW, test_idx_SW = sampleWindows(x_test_cv[:, 0:500, :, :], y_test_cv, 250, 20)

In [ ]:
x_train_SW = conv2DPrepreprocessing(x_train_SW, 5)

x_val_SW = conv2DPrepreprocessing(x_val_SW, 5)

x_test_SW = conv2DPrepreprocessing(x_test_SW, 5)

In [ ]:
x_train_SW.shape

(33840, 5, 50, 22)

In [ ]:
# Building the CNN model using sequential class
tf.keras.backend.clear_session()
CNN2D = Sequential()

# Conv. block 1
CNN2D.add(Conv2D(filters=64, kernel_size=(3,7), padding='same', input_shape=(5, 50, 22),activation='elu'))
# basic_cnn_model2d.add(tensorflow.keras.layers.PReLU())
CNN2D.add(MaxPooling2D(pool_size=(2,2), padding='same')) # Read the keras documentation
CNN2D.add(BatchNormalization())
CNN2D.add(Dropout(0.5))

# Conv. block 2
CNN2D.add(Conv2D(filters=64, kernel_size=(3,7), padding='same',activation='elu'))
# basic_cnn_model2d.add(tensorflow.keras.layers.PReLU())
CNN2D.add(MaxPooling2D(pool_size=(2,2), padding='same'))
CNN2D.add(BatchNormalization())
CNN2D.add(Dropout(0.5))

# Conv. block 3
CNN2D.add(Conv2D(filters=128, kernel_size=(3,7), padding='same',activation='elu'))
# basic_cnn_model2d.add(tensorflow.keras.layers.PReLU())
CNN2D.add(MaxPooling2D(pool_size=(2,2), padding='same'))
CNN2D.add(BatchNormalization())
CNN2D.add(Dropout(0.5))

# Conv. block 4
CNN2D.add(Conv2D(filters=256, kernel_size=(3,7), padding='same',activation='elu'))
# basic_cnn_model2d.add(tensorflow.keras.layers.PReLU())
CNN2D.add(MaxPooling2D(pool_size=(2,2), padding='same'))
CNN2D.add(BatchNormalization())
CNN2D.add(Dropout(0.5))

# Output layer with Softmax activation
CNN2D.add(Flatten()) # Flattens the input
CNN2D.add(Dense(4, activation='softmax')) # Output FC layer with softmax activation


# Printing the model summary
CNN2D.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 5, 50, 64)         29632     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3, 25, 64)        0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 3, 25, 64)        256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 3, 25, 64)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 25, 64)         86080     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2, 13, 64)        0

In [ ]:
# Model parameters
learning_rate = 1e-4
epochs = 50
cnn_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
# Compiling the model
learning_rate = 2e-5
cnn_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
epochs = 50
CNN2D.compile(loss=categorical_crossentropy,
                 optimizer=cnn_optimizer,
                 metrics=['accuracy'])

# Training and validating the model
CNN2D_results = CNN2D.fit(x_train_SW,
             y_train_SW,
             batch_size=128,
             epochs=epochs,
             validation_data=(x_val_SW, y_val_SW), verbose=True)

Epoch 1/50
265/265 [==============================] - 11s 35ms/step - loss: 0.6864 - accuracy: 0.7270 - val_loss: 1.0000 - val_accuracy: 0.5957
Epoch 2/50
265/265 [==============================] - 9s 33ms/step - loss: 0.6799 - accuracy: 0.7305 - val_loss: 1.0181 - val_accuracy: 0.5897
Epoch 3/50
265/265 [==============================] - 9s 33ms/step - loss: 0.6758 - accuracy: 0.7335 - val_loss: 1.0076 - val_accuracy: 0.5983
Epoch 4/50
265/265 [==============================] - 9s 33ms/step - loss: 0.6758 - accuracy: 0.7311 - val_loss: 1.0126 - val_accuracy: 0.5975
Epoch 5/50
265/265 [==============================] - 9s 33ms/step - loss: 0.6677 - accuracy: 0.7352 - val_loss: 1.0107 - val_accuracy: 0.5948
Epoch 6/50
265/265 [==============================] - 9s 33ms/step - loss: 0.6605 - accuracy: 0.7367 - val_loss: 1.0055 - val_accuracy: 0.5994
Epoch 7/50
265/265 [==============================] - 9s 35ms/step - loss: 0.6639 - accuracy: 0.7370 - val_loss: 1.0106 - val_accuracy: 0.603

In [ ]:
cnn_score = CNN2D.evaluate(x_test_SW, y_test_SW, verbose=0)
print('Test accuracy of the basic CNN model:',cnn_score[1])

Test accuracy of the basic CNN model: 0.6084650158882141


In [ ]:
maxVoting(x_test_SW, y_test_SW, test_idx_SW, x_test_cv, CNN2D)

0.6591422121896162
